#Import du Grand livre#

In [20]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

## Charger les variables d'environnement ##

In [21]:
# Charger le .env depuis la racine du projet
load_result = load_dotenv(dotenv_path='../../.env')
print(f"Chargement .env : {'✓ Réussi' if load_result else '✗ Échec'}")

Chargement .env : ✓ Réussi


## Connexion Redshift Pennylane ##

In [22]:
conn = psycopg2.connect(
    host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
    port=5439,
    dbname='prod',
    user='u_289572',
    password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
)

print("Connexion Redshift etablie")

Connexion Redshift etablie


## Fonction helper : Connexion auto-reconnect ##

In [23]:
def get_active_connection():
    """Retourne une connexion active, la recrée si nécessaire"""
    global conn
    try:
        # Test si la connexion est active
        pd.read_sql("SELECT 1;", conn)
        return conn
    except:
        # Reconnexion si fermée
        print("⟳ Reconnexion Redshift...")
        conn = psycopg2.connect(
            host='pennylane-external.csqwamh5pldr.eu-west-1.redshift.amazonaws.com',
            port=5439,
            dbname='prod',
            user='u_289572',
            password=os.getenv('PENNYLANE_DATA_SHARING_KEY')
        )
        print("✓ Reconnecté")
        return conn

# Utilisation : remplace "conn" par "get_active_connection()" dans tes requêtes
# Exemple :
df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())
print(f"Test réussi : {len(df)} ligne(s)")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_38764\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_38764\3220976796.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM pennylane.customers LIMIT 1;", get_active_connection())


Test réussi : 1 ligne(s)


## Liste table Grand Livre Pennylane ##

In [24]:
# Tables Grand Livre Pennylane
tables_extended = [
    'general_ledger'
]

accessible_tables = []

for table in tables_extended:
    try:
        query = f"SELECT * FROM pennylane.{table} LIMIT 1;"
        df = pd.read_sql(query, conn)
        accessible_tables.append({
            'table_name': table,
            'columns': len(df.columns),
            'column_list': ', '.join(df.columns.tolist()[:5]) + '...'
        })
        print(f"✓ {table} ({len(df.columns)} colonnes)")
    except:
        pass

print(f"\n\n=== RÉSUMÉ: {len(accessible_tables)} tables accessibles ===")
df_accessible = pd.DataFrame(accessible_tables)
print(df_accessible.to_string(index=False))

C:\Users\ycloa\AppData\Local\Temp\ipykernel_38764\248391769.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✓ general_ledger (23 colonnes)


=== RÉSUMÉ: 1 tables accessibles ===
    table_name  columns                                      column_list
general_ledger       23 company_id, company_name, id, date, lettering...


## Liste colonnes Grand Livre Pennylane ##

In [25]:
# Afficher les colonnes grand livre
tables_found = [
    'general_ledger'
]

for table in tables_found:
    query = f"SELECT * FROM pennylane.{table} LIMIT 0;"  # LIMIT 0 = structure seulement
    df = pd.read_sql(query, conn)
    print(f"\n{'='*60}")
    print(f"Table: pennylane.{table}")
    print(f"{'='*60}")
    print(f"Colonnes ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")


Table: pennylane.general_ledger
Colonnes (23):
   1. company_id
   2. company_name
   3. id
   4. date
   5. lettering
   6. label
   7. debit
   8. credit
   9. plan_item_number
  10. plan_item_label
  11. journal_code
  12. journal_label
  13. document_id
  14. document_label
  15. invoice_number
  16. fec_pieceref
  17. thirdparty_id
  18. thirdparty_plan_item_number
  19. thirdparty_plan_item_label
  20. invoice_link
  21. created_by
  22. document_created_at
  23. document_updated_at


C:\Users\ycloa\AppData\Local\Temp\ipykernel_38764\2518381739.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Chargement des données du grand livre ##

In [26]:
# Charger le grand livre complet
query_gl = "SELECT * FROM pennylane.general_ledger;"
df_general_ledger = pd.read_sql(query_gl, get_active_connection())

print(f"✓ Données chargées : {len(df_general_ledger)} lignes, {len(df_general_ledger.columns)} colonnes")

C:\Users\ycloa\AppData\Local\Temp\ipykernel_38764\3220976796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT 1;", conn)
C:\Users\ycloa\AppData\Local\Temp\ipykernel_38764\867651531.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_general_ledger = pd.read_sql(query_gl, get_active_connection())


✓ Données chargées : 2230 lignes, 23 colonnes


## Affichage des colonnes disponibles ##

In [27]:
print("Colonnes disponibles :")
for i, col in enumerate(df_general_ledger.columns, 1):
    print(f"  {i:2d}. {col}")

Colonnes disponibles :
   1. company_id
   2. company_name
   3. id
   4. date
   5. lettering
   6. label
   7. debit
   8. credit
   9. plan_item_number
  10. plan_item_label
  11. journal_code
  12. journal_label
  13. document_id
  14. document_label
  15. invoice_number
  16. fec_pieceref
  17. thirdparty_id
  18. thirdparty_plan_item_number
  19. thirdparty_plan_item_label
  20. invoice_link
  21. created_by
  22. document_created_at
  23. document_updated_at


## Suppresion de colonnes ##

In [28]:
# Colonnes à supprimer
columns_to_drop = ["id", "company_id", "invoice_link", "document_created_at", "document_updated_at"]

# Vérifier quelles colonnes existent réellement
existing_cols = [col for col in columns_to_drop if col in df_general_ledger.columns]
missing_cols = [col for col in columns_to_drop if col not in df_general_ledger.columns]

print(f"Colonnes supprimées : {existing_cols}")
if missing_cols:
    print(f"Colonnes inexistantes (ignorées) : {missing_cols}")

# Supprimer les colonnes
df_general_ledger = df_general_ledger.drop(columns=existing_cols)

print(f"\n✓ Résultat : {len(df_general_ledger.columns)} colonnes restantes")
print(f"Colonnes finales :\n{df_general_ledger.columns.tolist()}")

Colonnes supprimées : ['id', 'company_id', 'invoice_link', 'document_created_at', 'document_updated_at']

✓ Résultat : 18 colonnes restantes
Colonnes finales :
['company_name', 'date', 'lettering', 'label', 'debit', 'credit', 'plan_item_number', 'plan_item_label', 'journal_code', 'journal_label', 'document_id', 'document_label', 'invoice_number', 'fec_pieceref', 'thirdparty_id', 'thirdparty_plan_item_number', 'thirdparty_plan_item_label', 'created_by']


## Affichage type clonne ##

In [29]:
print(f"Types de données des {len(df_general_ledger.columns)} colonnes :\n")
for i, (col, dtype) in enumerate(df_general_ledger.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

Types de données des 18 colonnes :

   1. company_name                   → object
   2. date                           → object
   3. lettering                      → float64
   4. label                          → object
   5. debit                          → float64
   6. credit                         → float64
   7. plan_item_number               → object
   8. plan_item_label                → object
   9. journal_code                   → object
  10. journal_label                  → object
  11. document_id                    → int64
  12. document_label                 → object
  13. invoice_number                 → object
  14. fec_pieceref                   → object
  15. thirdparty_id                  → float64
  16. thirdparty_plan_item_number    → object
  17. thirdparty_plan_item_label     → object
  18. created_by                     → object


## Conversion des types de colonnes ##

In [30]:
# Conversion des types
df_general_ledger = df_general_ledger.astype({
    'date': 'datetime64[ns]',
    'debit': 'float64',
    'credit': 'float64'
})

# Convertir toutes les autres colonnes en string
columns_to_string = [col for col in df_general_ledger.columns if col not in ['date', 'debit', 'credit']]
df_general_ledger[columns_to_string] = df_general_ledger[columns_to_string].astype(str)

print("✓ Conversion effectuée\n")
print("Nouveaux types :")
for i, (col, dtype) in enumerate(df_general_ledger.dtypes.items(), 1):
    print(f"  {i:2d}. {col:<30} → {dtype}")

✓ Conversion effectuée

Nouveaux types :
   1. company_name                   → object
   2. date                           → datetime64[ns]
   3. lettering                      → object
   4. label                          → object
   5. debit                          → float64
   6. credit                         → float64
   7. plan_item_number               → object
   8. plan_item_label                → object
   9. journal_code                   → object
  10. journal_label                  → object
  11. document_id                    → object
  12. document_label                 → object
  13. invoice_number                 → object
  14. fec_pieceref                   → object
  15. thirdparty_id                  → object
  16. thirdparty_plan_item_number    → object
  17. thirdparty_plan_item_label     → object
  18. created_by                     → object


## Ajout des colonnes PCG (Plan Comptable Général) ##

In [31]:
# Extraction des niveaux du plan comptable
df_general_ledger['PCG_3'] = df_general_ledger['plan_item_number'].astype(str).str[:3]
df_general_ledger['PCG_2'] = df_general_ledger['plan_item_number'].astype(str).str[:2]
df_general_ledger['PCG_1'] = df_general_ledger['plan_item_number'].astype(str).str[:1]

print("✓ Colonnes PCG ajoutées\n")

# Vérification avec quelques exemples
print("Aperçu des extractions :")
print(df_general_ledger[['plan_item_number', 'PCG_1', 'PCG_2', 'PCG_3']].head(10))

print(f"\n✓ Total colonnes : {len(df_general_ledger.columns)}")

✓ Colonnes PCG ajoutées

Aperçu des extractions :
  plan_item_number PCG_1 PCG_2 PCG_3
0         401OFFIC     4    40   401
1         401OFFIC     4    40   401
2             6064     6    60   606
3             6064     6    60   606
4             6227     6    62   622
5         401GREFF     4    40   401
6             6227     6    62   622
7         401LEGAL     4    40   401
8           445662     4    44   445
9           445662     4    44   445

✓ Total colonnes : 21


## Ajout Nature du compte ##

In [32]:
# Créer la colonne Nature Compte basée sur PCG_1 (format texte)
df_general_ledger['Nature_Compte'] = df_general_ledger['PCG_1'].apply(
    lambda x: 'Resultat' if x in ['6', '7'] else 'Bilan'
).astype(str)

print("✓ Colonne 'Nature_Compte' ajoutée (format texte)\n")

# Vérification du type
print(f"Type de la colonne : {df_general_ledger['Nature_Compte'].dtype}")

# Vérification de la répartition
print("\nRépartition par nature de compte :")
print(df_general_ledger['Nature_Compte'].value_counts())

print("\nAperçu :")
print(df_general_ledger[['plan_item_number', 'PCG_1', 'Nature_Compte']].head(10))

✓ Colonne 'Nature_Compte' ajoutée (format texte)

Type de la colonne : object

Répartition par nature de compte :
Nature_Compte
Bilan       1753
Resultat     477
Name: count, dtype: int64

Aperçu :
  plan_item_number PCG_1 Nature_Compte
0         401OFFIC     4         Bilan
1         401OFFIC     4         Bilan
2             6064     6      Resultat
3             6064     6      Resultat
4             6227     6      Resultat
5         401GREFF     4         Bilan
6             6227     6      Resultat
7         401LEGAL     4         Bilan
8           445662     4         Bilan
9           445662     4         Bilan


## Ajout de la colonne Solde ##

In [33]:
# Créer la colonne Solde basée sur Nature_Compte
df_general_ledger['Solde'] = df_general_ledger.apply(
    lambda row: row['debit'] - row['credit'] if row['Nature_Compte'] == 'Bilan' 
                else row['credit'] - row['debit'],
    axis=1
).astype(float)

print("✓ Colonne 'Solde' ajoutée (format nombre)\n")

# Vérification du type
print(f"Type de la colonne : {df_general_ledger['Solde'].dtype}")

# Statistiques sur le solde
print("\nStatistiques sur la colonne Solde :")
print(df_general_ledger['Solde'].describe())

print("\nAperçu :")
print(df_general_ledger[['Nature_Compte', 'debit', 'credit', 'Solde']].head(10))

✓ Colonne 'Solde' ajoutée (format nombre)

Type de la colonne : float64

Statistiques sur la colonne Solde :
count     2230.000000
mean         3.673354
std       1647.165441
min     -10021.000000
25%        -21.990000
50%         -7.600000
75%          5.620000
max      10000.000000
Name: Solde, dtype: float64

Aperçu :
  Nature_Compte   debit  credit   Solde
0         Bilan    0.00   44.30  -44.30
1         Bilan    0.00   36.66  -36.66
2      Resultat   36.92    0.00  -36.92
3      Resultat   30.54    0.00  -30.54
4      Resultat  146.44    0.00 -146.44
5         Bilan    0.00   11.23  -11.23
6      Resultat    9.35    0.00   -9.35
7         Bilan    0.00  175.73 -175.73
8         Bilan   29.29    0.00   29.29
9         Bilan    1.88    0.00    1.88


## Connexion PostgreSQL local ##

In [34]:
# Connexion à PostgreSQL local (utilise les variables du .env)
conn_pg = psycopg2.connect(
    host=os.getenv('POSTGRES_HOST'),
    port=int(os.getenv('POSTGRES_PORT')),
    dbname=os.getenv('POSTGRES_DB'),
    user=os.getenv('POSTGRES_USER'),
    password=os.getenv('POSTGRES_PASSWORD')
)

print("✓ Connexion PostgreSQL établie")
print(f"  Database: {os.getenv('POSTGRES_DB')}")
print(f"  User: {os.getenv('POSTGRES_USER')}")
print(f"  Port: {os.getenv('POSTGRES_PORT')}")

✓ Connexion PostgreSQL établie
  Database: pennylane_data
  User: pennylane_user
  Port: 5433


## Import des données dans PostgreSQL avec SQLAlchemy ##

In [35]:
from sqlalchemy import create_engine

# Créer un engine SQLAlchemy
engine = create_engine(
    f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST')}:{os.getenv('POSTGRES_PORT')}/{os.getenv('POSTGRES_DB')}"
)

print("✓ Engine SQLAlchemy créé")

✓ Engine SQLAlchemy créé


## Export du DataFrame vers PostgreSQL ##